<center > <span style = "background-color: rgb(251, 160, 38); font-size: 32px;" > <b > NUCLIO DIGITAL SCHOOL - </b > MASTER EN DATA SCIENCE < /span > </center >

<br >

<center > <a href = https: // nuclio.school/wp-content/uploads/2019/10/nucleoDS-newBlack.png > <img src = "https://nuclio.school/wp-content/uploads/2019/10/nucleoDS-newBlack.png" width = 400 height = 100 > <a/>


<hr style="height:5px;border-width:0;color:orange;background-color:orange">
<center> <span style="font-size: 26px;"> Machine Learning for Time Series </span> </center>

<span style="font-size: 16px;">

+ Session: **Machine Learning for Time Series**
+ Module: **Time Series Forecasting**
+ Course: **Data Science Master 0921**
+ Professor: **Christa Santos**

</span>

<br>

<hr style="height:5px;border-width:0;color:orange;background-color:orange">

In this section, we will prepare the data for machine learning analysis by creating lagged variables. We will work with both stationary and non-stationary data. Three of the datasets that we are using are seasonally adjusted for stationarity through differencing. The two financial datasets are only resampled from daily to monthly data.  We will create kernel density and autocorrelation plots as well as generate lag variables for a 12-time step period. We will then save these new dataframes.

In [ ]:
import pandas as pd
from pandas import read_csv
from matplotlib import pyplot as plt

Let's look at the vacation dataset from the previous modules.

In [ ]:
# Read in data as a panda series
# https://trends.google.com/trends/?geo=US , google trends, search the word "vacation"
# Recall that data is from 2004 to 2019
vacation = pd.read_csv("diff_data/vacation_firstdiff.csv", index_col=0, parse_dates=True)


In [ ]:
# display first few rows
print(vacation.head(5))
# line plot of dataset
vacation.plot(figsize=(8, 5))
plt.show()
# data is monthly and is made stationary


The plot shows that seasonality and trend were removed from the series through differening. Above is the plot of the differenced data.

In [ ]:
vacation.describe()

In [ ]:
vacation.plot(kind='kde')


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
# Check for autocorrelation of each lagged observation and whether it is statistically significant.
plot_acf(vacation)
plt.show()


The blue shaded region is the margin of uncertainty. Candlesticks that extend out beyond the blue shaded region are considered statistically significant.  Correlation values are between 1 and -1.

### Create Lagged Variables

In [ ]:
# Reframe as supervised learning
# lag observation (t-1) is the input variable and t is the output variable.
df1 = pd.DataFrame()
print(df1)


In [ ]:
# Create 12 months of lag values to predict current observation
# Shift of 12 months
for i in range(12, 0, -1):
    df1[['t-'+str(i)]] = vacation.shift(i)
print(df1)


In [ ]:
# Create column t
df1['t'] = vacation.values
print(df1.head(13))

In [ ]:
# Create a new subsetted dataframe, removing Nans from first 12 rows
df1_vacat = df1[13:]
print(df1_vacat)


In [ ]:
# save to new file
df1_vacat.to_csv('vacation_lags_12months_features.csv', index=False)

Repeat the same for the furniture data

In [ ]:
furniture = pd.read_csv("diff_data/furn_pctchange.csv", index_col=0, parse_dates=True)
furniture.head()


<hr style="height:5px;border-width:0;color:orange;background-color:orange">

### Decision Tree Model for Time Series

In [ ]:
# load data
df1 = pd.read_csv('vacation_lags_12months_features.csv', header=0)
df1.head()


In [ ]:
# Split data

vacat = df1.values
# split into lagged variables and original time series
# slice all rows and start with column 0 and go up to but not including the last column
X1 = vacat[:, 0:-1]
# slice all rows and last column, essentially separating out 't' column
y1 = vacat[:, -1]


In [ ]:
# Columns t-1 to t-12, which are the lagged variables
X1


In [ ]:
# Column t, which is the original time series
y1[0:10]


In [ ]:
# Target Train-Test split
from pandas import read_csv

Y1 = y1
traintarget_size = int(len(Y1) * 0.80)   # Set split
train_target, test_target = Y1[0:traintarget_size], Y1[traintarget_size:len(Y1)]

print('Observations for Target: %d' % (len(Y1)))
print('Training Observations for Target: %d' % (len(train_target)))
print('Testing Observations for Target: %d' % (len(test_target)))


In [ ]:
# Features Train-Test split

trainfeature_size = int(len(X1) * 0.80)
train_feature, test_feature = X1[0:trainfeature_size], X1[trainfeature_size:len(X1)]
print('Observations for feature: %d' % (len(X1)))
print('Training Observations for feature: %d' % (len(train_feature)))
print('Testing Observations for feature: %d' % (len(test_feature)))


In [ ]:
# Decision Tree Regresion Model

from sklearn.tree import DecisionTreeRegressor

# Create a decision tree regression model with default arguments
decision_tree_vacat = DecisionTreeRegressor()  # max-depth not set

# Fit the model to the training features and targets
decision_tree_vacat.fit(train_feature, train_target)

# Check the score on train and test
print(decision_tree_vacat.score(train_feature, train_target))
# predictions are horrible if negative value, no relationship if 0
print(decision_tree_vacat.score(test_feature, test_target))


In [ ]:
# Find the best Max Depth

# Loop through a few different max depths and check the performance
# Try different max depths. We want to optimize our ML models to make the best predictions possible.
# For regular decision trees, max_depth, which is a hyperparameter, limits the number of splits in a tree.
# You can find the best value of max_depth based on the R-squared score of the model on the test set.

for d in [2, 3, 4, 5, 7, 8, 10]:
    # Create the tree and fit it
    decision_tree_vacat = DecisionTreeRegressor(max_depth=d)
    decision_tree_vacat.fit(train_feature, train_target)

    # Print out the scores on train and test
    print('max_depth=', str(d))
    print(decision_tree_vacat.score(train_feature, train_target))
    # You want the test score to be positive and high
    print(decision_tree_vacat.score(test_feature, test_target), '\n')

Decision-tree learners can create over-complex trees that do not generalize the data well. This is called overfitting.

In [ ]:
# Plot predicted against actual values

from matplotlib import pyplot as plt

# Use the best max_depth
decision_tree_vacat = DecisionTreeRegressor(
    max_depth=8)  # fill in best max depth here
decision_tree_vacat.fit(train_feature, train_target)

# Predict values for train and test
train_prediction = decision_tree_vacat.predict(train_feature)
test_prediction = decision_tree_vacat.predict(test_feature)

# Scatter the predictions vs actual values
plt.scatter(train_prediction, train_target, label='train')  # blue
plt.scatter(test_prediction, test_target, label='test')  # orange
plt.show()


### Random Forest for Time Series

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestRegressor

# Create the random forest model and fit to the training data
# n_estimators is the number of trees in forest
rfr = RandomForestRegressor(n_estimators=200)
rfr.fit(train_feature, train_target)

# Look at the R^2 scores on train and test
print(rfr.score(train_feature, train_target))
print(rfr.score(test_feature, test_target))  # Try to attain a positive value


In [ ]:
# Note: This cell will take some time to run
#  https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
from sklearn.model_selection import ParameterGrid
import numpy as np

# Create a dictionary of hyperparameters to search
# n_estimators is the number of trees in the forest. The larger the better,
# but also the longer it will take to compute.
# max_features is the number of features chosen at random at splits
# max_depth is the number of splits
# random_state will make your result reproducible

# Run grid search
# grid = {'n_estimators': [200], 'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'random_state': [17]}
grid = {'n_estimators': [200], 'max_depth': [10],
        'max_features': [2], 'random_state': [17]}
test_scores = []

# Loop through the parameter grid, set the hyperparameters, and save the scores
for g in ParameterGrid(grid):
    rfr.set_params(**g)  # ** is "unpacking" the dictionary
    rfr.fit(train_feature, train_target)
    test_scores.append(rfr.score(test_feature, test_target))

# Find best hyperparameters from the test score and print
best_idx = np.argmax(test_scores)
print(test_scores[best_idx], ParameterGrid(grid)[
      best_idx])  # You don't want negative value

# The best test score


In [ ]:
# Use the best hyperparameters from before to fit a random forest model
rfr = RandomForestRegressor(
    n_estimators=200, max_depth=10, max_features=2, random_state=17)
rfr.fit(train_feature, train_target)

# Make predictions with our model
train_prediction = rfr.predict(train_feature)
test_prediction = rfr.predict(test_feature)

# Create a scatter plot with train and test actual vs predictions
plt.scatter(train_target, train_prediction, label='train')
plt.scatter(test_target, test_prediction, label='test')
plt.legend()
plt.show()


In [ ]:
# Random Forest Model Feature Importance

# get column names
df1.columns
# Get feature importances from our random forest model
importances = rfr.feature_importances_

# Get the index of importances from greatest importance to least
sorted_index = np.argsort(importances)[::-1]
x1 = range(len(importances))

# Create tick labels
feature_names = ['t-12', 't-11', 't-10', 't-9', 't-8', 't-7', 't-6', 't-5', 't-4', 't-3',
                 't-2', 't-1']
labels = np.array(feature_names)[sorted_index]
plt.bar(x1, importances[sorted_index], tick_label=labels)

# Rotate tick labels to vertical
plt.xticks(rotation=90)
plt.show()


### Gradient Boosted Regression Model

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html
from sklearn.ensemble import GradientBoostingRegressor


# The fraction of samples to be used for fitting the individual base learners.
# Choosing subsample < 1.0 leads to a reduction of variance and an increase in bias.
# Create GB model -- hyperparameters
gbr = GradientBoostingRegressor(max_features=2,
                                learning_rate=0.01,
                                n_estimators=500,
                                subsample=0.6,
                                random_state=99)

gbr.fit(train_feature, train_target)

print(gbr.score(train_feature, train_target))
print(gbr.score(test_feature, test_target))


In [ ]:
# Gradient Boosted Model Feature Importance
# Extract feature importances from the fitted gradient boosting model
feature_importances = gbr.feature_importances_

# Get the indices of the largest to smallest feature importances
sorted_index = np.argsort(feature_importances)[::-1]
x1 = range(X1.shape[1])

# Create tick labels
feature_names = ['t-12', 't-11', 't-10', 't-9', 't-8', 't-7', 't-6', 't-5', 't-4', 't-3',
                 't-2', 't-1']
labels = np.array(feature_names)[sorted_index]

plt.bar(x1, feature_importances[sorted_index], tick_label=labels)

# Set the tick lables to be the feature names, according to the sorted feature_idx
plt.xticks(rotation=90)
plt.show()